In [1]:
import os
from os import path
import pandas as pd
import sys
basepath = path.join(os.getcwd(), "export_us")

In [2]:
import os
import numpy as np
import pandas as pd

start, end = '2015-06-30', '2024-06-27'
stock_codes = pd.read_csv('sp500_companies.csv')['Symbol']

dfs = []
for stock_code in stock_codes:
    fname = f"{stock_code}.csv"
    fpath = os.path.join(basepath, fname)
    if os.path.exists(fpath):
        df = pd.read_csv(fpath, parse_dates=['日期'])
        df.rename(columns={'日期': 'date', '开盘': 'open', '收盘': 'close', '最高': 'high', '最低': 'low', '成交量': 'volume', '成交额': 'amount'}, inplace=True)
        df['stock_code'] = stock_code
        dfs.append(df[(df['date'] >= start) & (df['date'] <= end)])

all_df = pd.concat(dfs, ignore_index=True)
all_df.set_index(['stock_code', 'date'], inplace=True)
all_df.sort_index(inplace=True)

In [3]:
all_df

open   close    high     low    volume       amount  \
stock_code date                                                                
AAPL       2015-06-30  23.203  23.165  23.340  23.025  43794410          0.0   
           2015-07-01  23.535  23.440  23.545  23.308  30028055          0.0   
           2015-07-02  23.418  23.410  23.483  23.253  26997705          0.0   
           2015-07-06  23.045  23.310  23.368  23.023  27844079          0.0   
           2015-07-07  23.283  23.233  23.348  22.753  46692274          0.0   
...                       ...     ...     ...     ...       ...          ...   
XEL        2024-06-21  50.975  50.565  51.345  50.395   6889808  368974432.0   
           2024-06-24  50.805  51.175  51.525  50.515   2705293  146002122.0   
           2024-06-25  51.075  50.935  51.145  50.555   2312056  124133156.0   
           2024-06-26  50.755  50.985  51.060  50.315   2662913  142969065.0   
           2024-06-27  51.035  51.045  51.340  50.655   2471025  133086875.0   

                         振幅   涨跌幅    涨跌额   换手率  
stock_code date                                 
AAPL       2015-06-30  1.37  0.95  0.217  0.77  
           2015-07-01  1.02  1.19  0.275  0.53  
           2015-07-02  0.98 -0.13 -0.030  0.47  
           2015-07-06  1.47 -0.43 -0.100  0.49  
           2015-07-07  2.55 -0.33 -0.077  0.82  
...                     ...   ...    ...   ...  
XEL        2024-06-21  1.87 -0.32 -0.160  1.24  
           2024-06-24  2.00  1.21  0.610  0.49  
           2024-06-25  1.15 -0.47 -0.240  0.42  
           2024-06-26  1.46  0.10  0.050  0.48  
           2024-06-27  1.34  0.12  0.060  0.44  

[170786 rows x 10 columns]

In [4]:
all_df.to_csv("all_stock_data.csv")

In [5]:
all_df_loaded = pd.read_csv("all_stock_data.csv", parse_dates=['date'])
all_df_loaded.set_index(['stock_code', 'date'], inplace=True)
all_df_loaded.sort_index(inplace=True)

In [6]:
import pandas as pd
import numpy as np
from scipy.stats import spearmanr
from scipy.stats import pearsonr


# 读取合并后的数据
all_df_loaded = pd.read_csv("all_stock_data.csv", parse_dates=['date'])
all_df_loaded.set_index(['stock_code', 'date'], inplace=True)
all_df_loaded.sort_index(inplace=True)


# 定义你的因子函数，输入为单个股票的DataFrame
def factor_func(df, lookback_period_vwma=20, lookback_period_momentum=10, market_regime='Neutral'):
    # Calculate the difference between today's close and yesterday's close
    df['close_diff'] = df['close'].diff()
    
    # Calculate the average of the high and low price
    df['avg_price'] = (df['high'] + df['low']) / 2
    
    # Calculate the volume weighted by the relative close price
    df['rel_close'] = (df['close'] - df['open']) / df['open']
    df['vol_weighted_rel_close'] = df['rel_close'] * df['volume']
    
    # The heuristic factor is the sum of the close difference and the volume-weighted relative close
    factor = df['close_diff'] + df['vol_weighted_rel_close']
    
    return factor

# 计算每只股票的因子值
all_df_loaded['factor'] = all_df_loaded.groupby('stock_code').apply(lambda g: factor_func(g)).reset_index(level=0, drop=True)

# 计算未来6日收益率
all_df_loaded['future_return_6d'] = all_df_loaded.groupby('stock_code')['close'].shift(-6) / all_df_loaded['close'] - 1

# 取所有日期
start_date = pd.Timestamp('2021-01-01')
end_date = pd.Timestamp('2024-01-01')
all_dates = all_df_loaded.index.get_level_values('date').unique()
all_dates = all_dates[(all_dates >= start_date) & (all_dates <= end_date)]
ic_values = []

for date in all_dates:
    daily = all_df_loaded.xs(date, level='date')
    factors = daily['factor']
    returns = daily['future_return_6d']
    mask = factors.notna() & returns.notna()
    if mask.sum() >= 10:
        ic, _ = pearsonr(factors[mask], returns[mask])
        if not np.isnan(ic):
            ic_values.append(ic)

mean_ic = np.mean(ic_values)
print(f"Mean IC: {mean_ic:.10f}")

Mean IC: 0.0077561156
